# SAE Project - Session 2: Motion Planning and Trajectory Generation

## Session Objectives (4 hours)

1. Implement inverse kinematics for key positions
2. Design pick-and-place motion sequences
3. Generate smooth joint-space and Cartesian trajectories
4. Optimize motion for cycle time reduction

---

## Part 2.1: Inverse Kinematics Implementation

### Background

For the quality control task, we need to compute joint configurations for:
- Picking components from the incoming tray (9 positions in 3×3 grid)
- Moving to vision inspection station
- Placing in appropriate bins (green/yellow/red)

The OpenManipulator-X is a 4-DOF manipulator, which means:
- We can control position (X, Y, Z) and one orientation angle (typically yaw)
- The gripper orientation (roll, pitch) is constrained by the robot structure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from roboticstoolbox import DHRobot, RevoluteDH
from spatialmath import SE3, SO3
import roboticstoolbox as rtb
from scipy.interpolate import CubicSpline
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

%matplotlib inline

In [ ]:
# Load robot model from Session 1
# TODO: Copy your robot model implementation from Session 1

class OpenManipulatorX(DHRobot):
    def __init__(self):
        # Link dimensions
        d1 = 0.077
        a2 = 0.130
        a3 = 0.124
        a4 = 0.126
        
        L = [
            RevoluteDH(d=d1, a=0, alpha=np.pi/2),
            RevoluteDH(d=0, a=a2, alpha=0),
            RevoluteDH(d=0, a=a3, alpha=0),
            RevoluteDH(d=0, a=a4, alpha=0),
        ]
        
        super().__init__(L, name="OpenManipulator-X")
        
        self.qlim = np.array([
            [-np.pi, np.pi],
            [-np.pi/2, np.pi/2],
            [-80*np.pi/180, 95*np.pi/180],
            [-np.pi/2, np.pi/2]
        ])

robot = OpenManipulatorX()
print(robot)

## Task 2.1: Analytical IK for OpenManipulator-X

**Instructions:**
1. Derive the analytical inverse kinematics solution for the robot
2. The robot has a simple geometry (3R planar arm + base rotation)
3. Write equations for all joint angles

**Hints:**
- Joint 1 (θ₁): Base rotation = atan2(y, x)
- Joints 2, 3, 4: Form a planar 3R chain in the vertical plane
- Use geometric approach or algebra

**✍️ Hand-written derivation required in your report**

In [ ]:
# Task 2.1: Implement analytical inverse kinematics

def inverse_kinematics_analytical(target_pos, target_yaw=0, elbow_up=True):
    """
    Analytical IK for OpenManipulator-X
    
    Args:
        target_pos: [x, y, z] target position in meters
        target_yaw: Desired yaw angle (rotation about Z)
        elbow_up: True for elbow-up configuration, False for elbow-down
    
    Returns:
        q: [q1, q2, q3, q4] joint angles in radians, or None if unreachable
    """
    x, y, z = target_pos
    
    # Robot dimensions
    d1 = 0.077
    a2 = 0.130
    a3 = 0.124
    a4 = 0.126
    
    # TODO: Implement analytical IK
    # Step 1: Compute q1 (base rotation)
    # q1 = atan2(y, x)
    
    # Step 2: Project to 2D plane (R, Z)
    # R = sqrt(x^2 + y^2)
    # Account for tool offset a4
    
    # Step 3: Solve planar 2R problem for q2 and q3
    # Use law of cosines
    
    # Step 4: Compute q4 to achieve desired end-effector orientation
    
    return None  # Placeholder

# Test the IK function
test_position = [0.2, 0.15, 0.1]
q_analytical = inverse_kinematics_analytical(test_position)

if q_analytical is not None:
    print(f"Target position: {test_position}")
    print(f"Joint solution: {np.rad2deg(q_analytical)} degrees")
    
    # Verify with forward kinematics
    T = robot.fkine(q_analytical)
    print(f"FK verification: {T.t}")
    print(f"Position error: {np.linalg.norm(T.t - test_position):.6f} m")

## Task 2.2: Numerical IK using robotics-toolbox

Compare your analytical solution with numerical methods

In [ ]:
# Task 2.2: Numerical IK comparison

def inverse_kinematics_numerical(robot, target_pos, target_orient=None, q0=None):
    """
    Numerical IK using Levenberg-Marquardt
    
    Args:
        robot: Robot model
        target_pos: [x, y, z] target position
        target_orient: Optional SE3 or RPY orientation
        q0: Initial guess for joint angles
    
    Returns:
        q: Joint configuration, or None if failed
    """
    # Build target transformation
    if target_orient is None:
        # Default: gripper pointing down
        Tep = SE3(target_pos[0], target_pos[1], target_pos[2]) @ SE3.RPY([0, np.pi, 0])
    else:
        Tep = SE3(target_pos[0], target_pos[1], target_pos[2]) @ target_orient
    
    # Initial guess
    if q0 is None:
        q0 = [0, 0, 0, 0]
    
    # Solve IK
    sol = robot.ikine_LM(Tep, q0=q0)
    
    if sol.success:
        return sol.q
    else:
        return None

# Compare methods
test_positions = [
    [0.200, 0.000, 0.100],  # Front
    [0.150, 0.150, 0.080],  # Right-front
    [-0.100, 0.180, 0.120], # Left-front
]

for pos in test_positions:
    print(f"\nTarget: {pos}")
    
    # Analytical
    q_ana = inverse_kinematics_analytical(pos)
    if q_ana is not None:
        T_ana = robot.fkine(q_ana)
        error_ana = np.linalg.norm(T_ana.t - pos)
        print(f"  Analytical: {np.rad2deg(q_ana)}, error = {error_ana:.6f} m")
    
    # Numerical
    q_num = inverse_kinematics_numerical(robot, pos)
    if q_num is not None:
        T_num = robot.fkine(q_num)
        error_num = np.linalg.norm(T_num.t - pos)
        print(f"  Numerical:  {np.rad2deg(q_num)}, error = {error_num:.6f} m")

## Task 2.3: Generate Workspace Positions

Define all key positions for the quality control task

In [ ]:
# Task 2.3: Define workspace positions

class WorkspacePositions:
    """Define all key positions in the workcell"""
    
    def __init__(self):
        # Home position (safe position above workspace)
        self.home = [0.000, 0.000, 0.200]
        
        # Incoming tray: 3×3 grid of components
        # Grid spacing: 40mm between components
        self.tray_grid = self._generate_tray_grid(
            center=[0.000, -0.180, 0.050],
            spacing=0.040,
            rows=3,
            cols=3
        )
        
        # Vision inspection station
        self.vision_station = [0.000, 0.150, 0.080]
        
        # Sorting bins
        self.bins = {
            'GOOD': [-0.150, 0.200, 0.100],      # Green bin
            'REWORK': [0.000, 0.220, 0.100],     # Yellow bin
            'REJECT': [0.150, 0.200, 0.100],     # Red bin
        }
        
        # Approach height (offset above pick/place positions)
        self.approach_offset = 0.050  # 50mm above target
    
    def _generate_tray_grid(self, center, spacing, rows, cols):
        """Generate grid of positions for component tray"""
        positions = []
        x_center, y_center, z = center
        
        # Calculate offsets
        x_start = x_center - (cols - 1) * spacing / 2
        y_start = y_center - (rows - 1) * spacing / 2
        
        for i in range(rows):
            for j in range(cols):
                x = x_start + j * spacing
                y = y_start + i * spacing
                positions.append([x, y, z])
        
        return positions
    
    def get_approach_position(self, target_pos):
        """Get approach position above target"""
        return [target_pos[0], target_pos[1], target_pos[2] + self.approach_offset]

# Create workspace
workspace = WorkspacePositions()

print("Workspace positions defined:")
print(f"  Home: {workspace.home}")
print(f"  Tray grid: {len(workspace.tray_grid)} positions")
print(f"  Vision station: {workspace.vision_station}")
print(f"  Bins: {len(workspace.bins)} bins")

# Visualize positions
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot tray positions
tray_array = np.array(workspace.tray_grid)
ax.scatter(tray_array[:, 0], tray_array[:, 1], tray_array[:, 2], 
           c='blue', marker='o', s=100, label='Tray grid')

# Plot bins
colors = {'GOOD': 'green', 'REWORK': 'yellow', 'REJECT': 'red'}
for name, pos in workspace.bins.items():
    ax.scatter(pos[0], pos[1], pos[2], 
               c=colors[name], marker='s', s=200, label=f'{name} bin')

# Plot vision station
vs = workspace.vision_station
ax.scatter(vs[0], vs[1], vs[2], c='purple', marker='^', s=200, label='Vision')

# Plot home
h = workspace.home
ax.scatter(h[0], h[1], h[2], c='black', marker='*', s=200, label='Home')

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('Quality Control Workcell Layout')
ax.legend()
plt.show()

## Task 2.4: Joint-Space Trajectory Generation

**Objective:** Generate smooth trajectories in joint space for pick-and-place operations

**Requirements:**
- Use quintic (5th order) polynomials for smooth motion
- Zero velocity and acceleration at start and end
- Respect joint velocity and acceleration limits

In [ ]:
# Task 2.4: Joint-space trajectory planning

def quintic_polynomial(q0, qf, T, t):
    """
    Generate quintic polynomial trajectory
    
    Args:
        q0: Initial position
        qf: Final position
        T: Total time duration
        t: Time vector
    
    Returns:
        q: Position trajectory
        qd: Velocity trajectory
        qdd: Acceleration trajectory
    """
    # Quintic polynomial coefficients
    # q(t) = a0 + a1*t + a2*t^2 + a3*t^3 + a4*t^4 + a5*t^5
    # Boundary conditions: q(0)=q0, q(T)=qf, qd(0)=0, qd(T)=0, qdd(0)=0, qdd(T)=0
    
    # TODO: Compute coefficients
    a0 = q0
    a1 = 0
    a2 = 0
    a3 = 10 * (qf - q0) / T**3
    a4 = -15 * (qf - q0) / T**4
    a5 = 6 * (qf - q0) / T**5
    
    # Compute trajectory
    q = a0 + a1*t + a2*t**2 + a3*t**3 + a4*t**4 + a5*t**5
    qd = a1 + 2*a2*t + 3*a3*t**2 + 4*a4*t**3 + 5*a5*t**4
    qdd = 2*a2 + 6*a3*t + 12*a4*t**2 + 20*a5*t**3
    
    return q, qd, qdd

def generate_joint_trajectory(robot, q_start, q_end, duration=2.0, dt=0.01):
    """
    Generate multi-joint trajectory
    
    Args:
        robot: Robot model
        q_start: Initial joint configuration
        q_end: Final joint configuration
        duration: Motion duration in seconds
        dt: Time step
    
    Returns:
        t: Time vector
        Q: Joint position trajectory (N x n_joints)
        Qd: Joint velocity trajectory
        Qdd: Joint acceleration trajectory
    """
    t = np.arange(0, duration + dt, dt)
    n_joints = len(q_start)
    n_steps = len(t)
    
    Q = np.zeros((n_steps, n_joints))
    Qd = np.zeros((n_steps, n_joints))
    Qdd = np.zeros((n_steps, n_joints))
    
    # Generate trajectory for each joint
    for i in range(n_joints):
        Q[:, i], Qd[:, i], Qdd[:, i] = quintic_polynomial(
            q_start[i], q_end[i], duration, t
        )
    
    return t, Q, Qd, Qdd

# Test trajectory generation
q_home = [0, 0, 0, 0]
q_pick = inverse_kinematics_numerical(robot, workspace.tray_grid[0])

if q_pick is not None:
    t, Q, Qd, Qdd = generate_joint_trajectory(robot, q_home, q_pick, duration=2.0)
    
    # Plot trajectory
    fig, axes = plt.subplots(3, 1, figsize=(12, 8))
    joint_names = ['Joint 1', 'Joint 2', 'Joint 3', 'Joint 4']
    
    # Position
    for i in range(4):
        axes[0].plot(t, np.rad2deg(Q[:, i]), label=joint_names[i])
    axes[0].set_ylabel('Position (deg)')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_title('Joint-Space Trajectory')
    
    # Velocity
    for i in range(4):
        axes[1].plot(t, np.rad2deg(Qd[:, i]))
    axes[1].set_ylabel('Velocity (deg/s)')
    axes[1].grid(True)
    
    # Acceleration
    for i in range(4):
        axes[2].plot(t, np.rad2deg(Qdd[:, i]))
    axes[2].set_ylabel('Acceleration (deg/s²)')
    axes[2].set_xlabel('Time (s)')
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

## Task 2.5: Cartesian-Space Trajectory

For vision inspection, we need to move the component in a straight line through the vision system's field of view.

In [ ]:
# Task 2.5: Cartesian trajectory generation

def generate_cartesian_line(p_start, p_end, duration=1.0, dt=0.01):
    """
    Generate straight-line trajectory in Cartesian space
    
    Args:
        p_start: Starting position [x, y, z]
        p_end: End position [x, y, z]
        duration: Motion duration
        dt: Time step
    
    Returns:
        t: Time vector
        P: Position trajectory
        Pd: Velocity trajectory
    """
    t = np.arange(0, duration + dt, dt)
    n_steps = len(t)
    
    P = np.zeros((n_steps, 3))
    Pd = np.zeros((n_steps, 3))
    
    # Linear interpolation for each axis
    for i in range(3):
        P[:, i], Pd[:, i], _ = quintic_polynomial(
            p_start[i], p_end[i], duration, t
        )
    
    return t, P, Pd

def cartesian_to_joint_trajectory(robot, P, q0):
    """
    Convert Cartesian trajectory to joint trajectory using IK
    
    Args:
        robot: Robot model
        P: Cartesian position trajectory (N x 3)
        q0: Initial joint configuration
    
    Returns:
        Q: Joint trajectory (N x n_joints)
    """
    n_steps = P.shape[0]
    n_joints = robot.n
    Q = np.zeros((n_steps, n_joints))
    
    q_prev = q0
    for i, pos in enumerate(P):
        q = inverse_kinematics_numerical(robot, pos, q0=q_prev)
        if q is not None:
            Q[i, :] = q
            q_prev = q
        else:
            # If IK fails, keep previous configuration
            Q[i, :] = q_prev
    
    return Q

# Test Cartesian trajectory for vision inspection
# Move component slowly through vision field
vision_start = [workspace.vision_station[0] - 0.03, 
                workspace.vision_station[1], 
                workspace.vision_station[2]]
vision_end = [workspace.vision_station[0] + 0.03, 
              workspace.vision_station[1], 
              workspace.vision_station[2]]

t_cart, P_cart, Pd_cart = generate_cartesian_line(vision_start, vision_end, duration=1.5)

# Convert to joint space
q_init = inverse_kinematics_numerical(robot, vision_start)
Q_cart = cartesian_to_joint_trajectory(robot, P_cart, q_init)

# Visualize Cartesian path
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131)
ax1.plot(t_cart, P_cart[:, 0], label='X')
ax1.plot(t_cart, P_cart[:, 1], label='Y')
ax1.plot(t_cart, P_cart[:, 2], label='Z')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Position (m)')
ax1.set_title('Cartesian Position')
ax1.legend()
ax1.grid(True)

ax2 = fig.add_subplot(132)
ax2.plot(t_cart, np.linalg.norm(Pd_cart, axis=1))
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Speed (m/s)')
ax2.set_title('End-Effector Speed')
ax2.grid(True)

ax3 = fig.add_subplot(133, projection='3d')
ax3.plot(P_cart[:, 0], P_cart[:, 1], P_cart[:, 2], 'b-', linewidth=2)
ax3.scatter(P_cart[0, 0], P_cart[0, 1], P_cart[0, 2], c='green', s=100, label='Start')
ax3.scatter(P_cart[-1, 0], P_cart[-1, 1], P_cart[-1, 2], c='red', s=100, label='End')
ax3.set_xlabel('X (m)')
ax3.set_ylabel('Y (m)')
ax3.set_zlabel('Z (m)')
ax3.set_title('3D Path')
ax3.legend()

plt.tight_layout()
plt.show()

## Task 2.6: Complete Pick-and-Place Sequence

**Objective:** Plan the complete motion sequence for one component

**Sequence:**
1. Start at HOME
2. Move to APPROACH position above component in tray
3. Move down to PICK position
4. Close gripper (simulated)
5. Move up to APPROACH position
6. Move to VISION STATION approach
7. Scan through vision system (Cartesian line)
8. Based on quality result, move to appropriate BIN
9. Open gripper (simulated)
10. Return to HOME

In [ ]:
# Task 2.6: Complete pick-and-place motion planning

class PickPlaceMotionPlanner:
    """
    Motion planner for quality control pick-and-place operations
    """
    def __init__(self, robot, workspace):
        self.robot = robot
        self.workspace = workspace
        
        # Motion parameters
        self.t_fast = 1.5  # Fast motion time (s)
        self.t_slow = 1.0  # Slow motion time (s)
        self.t_pick_place = 0.5  # Pick/place motion time (s)
        self.t_vision = 1.5  # Vision scan time (s)
        
    def plan_complete_sequence(self, component_id, bin_type='GOOD'):
        """
        Plan complete motion sequence for one component
        
        Args:
            component_id: Index in tray (0-8)
            bin_type: 'GOOD', 'REWORK', or 'REJECT'
        
        Returns:
            trajectory: Dict with time and joint trajectories
        """
        # Define waypoints
        waypoints = []
        durations = []
        descriptions = []
        
        # 1. HOME position
        home_pos = self.workspace.home
        q_home = inverse_kinematics_numerical(self.robot, home_pos)
        waypoints.append(q_home)
        descriptions.append("HOME")
        
        # 2. Approach tray
        pick_pos = self.workspace.tray_grid[component_id]
        approach_pick = self.workspace.get_approach_position(pick_pos)
        q_approach_pick = inverse_kinematics_numerical(self.robot, approach_pick)
        waypoints.append(q_approach_pick)
        durations.append(self.t_fast)
        descriptions.append("APPROACH_TRAY")
        
        # 3. Pick position
        q_pick = inverse_kinematics_numerical(self.robot, pick_pos)
        waypoints.append(q_pick)
        durations.append(self.t_pick_place)
        descriptions.append("PICK")
        
        # 4. Lift from tray
        waypoints.append(q_approach_pick)
        durations.append(self.t_pick_place)
        descriptions.append("LIFT")
        
        # 5. Move to vision station
        vision_pos = self.workspace.vision_station
        q_vision = inverse_kinematics_numerical(self.robot, vision_pos)
        waypoints.append(q_vision)
        durations.append(self.t_fast)
        descriptions.append("VISION")
        
        # 6. Move to bin
        bin_pos = self.workspace.bins[bin_type]
        approach_bin = self.workspace.get_approach_position(bin_pos)
        q_approach_bin = inverse_kinematics_numerical(self.robot, approach_bin)
        waypoints.append(q_approach_bin)
        durations.append(self.t_fast)
        descriptions.append(f"APPROACH_{bin_type}_BIN")
        
        # 7. Place position
        q_place = inverse_kinematics_numerical(self.robot, bin_pos)
        waypoints.append(q_place)
        durations.append(self.t_pick_place)
        descriptions.append("PLACE")
        
        # 8. Lift from bin
        waypoints.append(q_approach_bin)
        durations.append(self.t_pick_place)
        descriptions.append("RELEASE")
        
        # 9. Return home
        waypoints.append(q_home)
        durations.append(self.t_fast)
        descriptions.append("RETURN_HOME")
        
        # Generate complete trajectory
        return self._generate_multi_waypoint_trajectory(
            waypoints, durations, descriptions
        )
    
    def _generate_multi_waypoint_trajectory(self, waypoints, durations, descriptions):
        """
        Generate trajectory through multiple waypoints
        """
        trajectory = {
            'time': [],
            'position': [],
            'velocity': [],
            'descriptions': descriptions,
            'segment_times': [0]
        }
        
        current_time = 0
        
        for i in range(len(waypoints) - 1):
            t, Q, Qd, _ = generate_joint_trajectory(
                self.robot, 
                waypoints[i], 
                waypoints[i+1], 
                duration=durations[i]
            )
            
            # Append to trajectory (skip first point to avoid duplicates)
            trajectory['time'].append(t[1:] + current_time)
            trajectory['position'].append(Q[1:, :])
            trajectory['velocity'].append(Qd[1:, :])
            
            current_time += durations[i]
            trajectory['segment_times'].append(current_time)
        
        # Concatenate all segments
        trajectory['time'] = np.concatenate(trajectory['time'])
        trajectory['position'] = np.vstack(trajectory['position'])
        trajectory['velocity'] = np.vstack(trajectory['velocity'])
        trajectory['total_time'] = current_time
        
        return trajectory

# Test the motion planner
planner = PickPlaceMotionPlanner(robot, workspace)
traj = planner.plan_complete_sequence(component_id=4, bin_type='GOOD')

print(f"Complete cycle time: {traj['total_time']:.2f} seconds")
print(f"\nMotion sequence:")
for i, desc in enumerate(traj['descriptions']):
    if i < len(traj['segment_times']) - 1:
        print(f"  {i}. {desc:20s} (t={traj['segment_times'][i]:.2f}s)")

# Plot trajectory
fig, ax = plt.subplots(2, 1, figsize=(14, 8))

# Joint positions
for i in range(4):
    ax[0].plot(traj['time'], np.rad2deg(traj['position'][:, i]), 
               label=f'Joint {i+1}')
ax[0].set_ylabel('Position (deg)')
ax[0].set_title('Complete Pick-and-Place Trajectory')
ax[0].legend()
ax[0].grid(True)

# Add segment markers
for t_seg in traj['segment_times'][1:-1]:
    ax[0].axvline(t_seg, color='r', linestyle='--', alpha=0.3)

# Joint velocities
for i in range(4):
    ax[1].plot(traj['time'], np.rad2deg(traj['velocity'][:, i]))
ax[1].set_ylabel('Velocity (deg/s)')
ax[1].set_xlabel('Time (s)')
ax[1].grid(True)

for t_seg in traj['segment_times'][1:-1]:
    ax[1].axvline(t_seg, color='r', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## Task 2.7: Cycle Time Analysis

Analyze the total cycle time and identify opportunities for optimization

In [ ]:
# Task 2.7: Cycle time analysis

def analyze_cycle_times(planner, n_components=9):
    """
    Analyze cycle times for all components and different quality outcomes
    """
    results = []
    
    bin_types = ['GOOD', 'REWORK', 'REJECT']
    
    for comp_id in range(min(n_components, len(workspace.tray_grid))):
        for bin_type in bin_types:
            traj = planner.plan_complete_sequence(comp_id, bin_type)
            results.append({
                'component': comp_id,
                'bin': bin_type,
                'cycle_time': traj['total_time']
            })
    
    # Analyze results
    import pandas as pd
    df = pd.DataFrame(results)
    
    print("Cycle Time Statistics:")
    print(df.groupby('bin')['cycle_time'].describe())
    
    # Plot cycle times
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for bin_type in bin_types:
        data = df[df['bin'] == bin_type]
        ax.plot(data['component'], data['cycle_time'], 
                marker='o', label=bin_type)
    
    ax.set_xlabel('Component Position in Tray')
    ax.set_ylabel('Cycle Time (s)')
    ax.set_title('Cycle Time vs Component Position')
    ax.legend()
    ax.grid(True)
    ax.axhline(y=8.0, color='r', linestyle='--', label='Target (<8s)')
    plt.show()
    
    return df

# Run analysis
# cycle_analysis = analyze_cycle_times(planner)

---

## Session 2 Deliverables

**By the end of Session 2, you should have:**

✅ Analytical and numerical inverse kinematics implementations  
✅ Complete workspace position definitions  
✅ Joint-space trajectory generation using quintic polynomials  
✅ Cartesian-space trajectory for vision scanning  
✅ Complete pick-and-place motion sequence  
✅ Cycle time analysis and optimization opportunities  

**Questions for Report:**

1. Compare analytical vs numerical IK: accuracy, computational cost, robustness
2. Why use quintic polynomials instead of cubic? What are the advantages?
3. Which components have the shortest/longest cycle times? Why?
4. How could you reduce cycle time while maintaining smooth motion?
5. What factors limit the maximum speed of motion?

---

**Continue to Session 3 notebook for quality inspection implementation...**